Import needed libraries:

In [34]:
import pandas as pd
import re
import os
import time
import reliability_and_inflation as ri 
import enrich_with_type_msrp as enrich
import google.generativeai as genai

only_alphanum = re.compile(r'[\W_]+')  #regular expression pattern to remove any non-alphanumeric characters
genai.configure(api_key = os.environ.get("GEMINI_API_KEY"))

# 1- Preparing the data:

Load datasets:

In [35]:
cars_train = pd.read_csv("train.csv")
cars_test = pd.read_csv("test.csv")

First, we use our Google Gemini-based model to add columns "body_style" and "original_msrp" to our data:

The parameters:

In [ ]:
model = genai.GenerativeModel("tunedModels/finetunedmodelcartypemsrpv1-19pkfyc97yiq", generation_config = {"temperature":0.1})
known_cars = pd.read_csv("list_of_examples.csv")
known_cars.rename(columns={"body_style":"body", "MSRP":"msrp"}, inplace=True)
batch_size = 60    #might have to be fixed based on tuned model
rounds = 5   #number of times each batch is asked about. Should be an odd number
possible_body_types = set(known_cars['body'].unique())
prompt_params = {"min_num":4, "max_num":8, "repeats":False,
                 "min_len":3, "max_len":10, "balance":True,
                 "type_size":None}  

We run the model:

In [ ]:
#Splitting into pieces so that we save after each piece
car_names = enrich.get_car_keys_names([cars_train, cars_test])
car_names_df = pd.DataFrame.from_dict(car_names, orient='index').reset_index()
total = len(car_names_df)
n=8
dfs = [car_names_df.iloc[(i*total)//n:((i+1)*total)//n] for i in range(n)]

for i in range(len(dfs)):
    dfs[i].to_csv("car_keys_slice_"+str(i+1)+".csv", index=False)



Total unique specific car models =  28070


In [ ]:
#car_names = enrich.get_car_keys_names([cars_train, cars_test])
for i in range(6, 9):   #Done 1-5. Need to do 6-8
    print(f"Working on slice {i}")
    read_dict = pd.read_csv(f"car_keys_slice_{i}.csv").to_dict('index')
    car_names = {read_dict[i]['index']:read_dict[i]['0'] for i in read_dict}
    car_body_types, car_msrps = enrich.enrich_using_model(model, car_names, known_cars, batch_size,
                                                            possible_body_types, prompt_params, 
                                                            rounds = rounds)


    car_body_types_df = pd.DataFrame.from_dict(car_body_types, orient='index').reset_index()
    car_msrps_df = pd.DataFrame.from_dict(car_msrps, orient='index').reset_index()

    car_body_types_df.to_csv(f"car_body_types_slice_{i}.csv", index=False)
    car_msrps_df.to_csv(f"car_msrps_slice_{i}.csv", index=False)

num answers = 60
num answers = 59
num answers = 60
num answers = 59
num answers = 57
num answers = 60
num answers = 59
num answers = 60
3480
num answers = 60
num answers = 60
num answers = 60
num answers = 60
num answers = 60
3509
Working on slice 7
Using genai.GenerativeModel(
    model_name='tunedModels/finetunedmodelcartypemsrpv1-19pkfyc97yiq',
    generation_config={'temperature': 0.1},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
):
Parameters:
Batch size = 60
Total number of car models = 3509
Min # of examples in each prompt = 4      Max # of examples in each prompt = 8
Min length of examples in each prompt = 3      Max length of examples in each prompt = 10
Allow repeats in each prompt example = False       Number of times each batch is asked about = 5

num answers = 59
num answers = 60
num answers = 60
num answers = 60
num answers = 59
num answers = 60
num answers = 60
60
num answers = 59
num answers = 60
num answers = 60
num answ

In [36]:

#combine results from slices
n= 8
car_body_types_read = pd.concat([pd.read_csv("car_body_types_slice_"+str(i+1)+".csv") for i in range(n)]).reset_index().to_dict('index')
car_msrps_read = pd.concat([pd.read_csv("car_msrps_slice_"+str(i+1)+".csv") for i in range(n)]).reset_index().to_dict('index')

car_body_types = {car_body_types_read[i]['index']:car_body_types_read[i]['0'] for i in car_body_types_read}
car_msrps = {car_msrps_read[i]['index']:car_msrps_read[i]['0'] for i in car_msrps_read}


We use the resulting data to add our new columns:

In [37]:
enrich.update_car_type_msrp([cars_train, cars_test], car_body_types, car_msrps)

Fill missing values in accident and clean_title columns:

In [38]:
missing_fill = {"accident": "unknown", "clean_title": "no", 
                "int_col": "unknown", "ext_col": "unknown",
                "engine": "unknown", "transmission": "unknown", "fuel_type": "unknown"}
cars_train.fillna(value=missing_fill, inplace=True)   #fill missing values
cars_test.fillna(value=missing_fill, inplace=True)

Remove any non-alphanumeric characters from columns with string input and change into lowercase:

In [39]:
cat_types = ["model", "brand", "ext_col", "int_col", "accident", 
             "clean_title", "full_name", "brand_model", "body_style",
             'engine','transmission','fuel_type']
for col in cat_types:
    cars_train[col] = cars_train[col].apply(lambda x: only_alphanum.sub("", x).lower())
    cars_test[col] = cars_test[col].apply(lambda x: only_alphanum.sub("", x).lower())

cars_train = cars_train.mask(cars_train == '')  #replace empty cells with NaN
cars_test = cars_test.mask(cars_test == '')
cars_train.fillna(value=missing_fill, inplace=True)   #refill missing values
cars_test.fillna(value=missing_fill, inplace=True)

Add "age" as a column:

In [40]:
cars_train["age"]=2025-cars_train["model_year"]
cars_test["age"]=2025-cars_test["model_year"]

Convert accident and clean_title into 0 or 1 values:

In [ ]:
cars_train['accident'] = cars_train['accident'].apply(lambda x: 0 if x == 'nonereported' else 1)
cars_test['accident'] = cars_test['accident'].apply(lambda x: 0 if x == 'nonereported' else 1)

cars_train['clean_title'] = cars_train['clean_title'].apply(lambda x: 0 if x == 'no' else 1)
cars_test['clean_title'] = cars_test['clean_title'].apply(lambda x: 0 if x == 'no' else 1)

Add a column "reliability" with a normalized reliability score based on brand:

In [41]:
reliability_dict = ri.build_reliability_dict()
cars_train['reliability'] = cars_train['brand'].apply(lambda x: ri.get_reliability(x, reliability_dict))
cars_test['reliability'] = cars_test['brand'].apply(lambda x: ri.get_reliability(x, reliability_dict))

We then add a column where we adjust the original msrp to 2024 dollars:

In [42]:
ri.adjust_for_inflation(cars_train, 'model_year', 'msrp', 2024)
ri.adjust_for_inflation(cars_test, 'model_year', 'msrp', 2024)

We add columns for the difference between the original and adjusted msrp on one hand and price on the other. These are to be tested as regression targets from which we can recover price.

In [43]:
cars_train['price_diff'] = cars_train['price']-cars_train['msrp']
cars_train['adjusted_price_diff'] = cars_train['price']-cars_train['adjusted_msrp']

We standardize color names a bit

In [44]:
def standardize_color(color, standard_colors = []):
    color = color.replace("grey", "gray")
    color = color.replace("obsidian", "black")
    color = color.replace("ebony", "black")
    color = color.replace("pearl", "white")
    for s_col in standard_colors:
        if color.find(s_col) != -1:
            return s_col
    return color

color_counts_ext = cars_train['ext_col'].value_counts()
standard_colors_ext = list(color_counts_ext[color_counts_ext>800].keys())
standard_colors_ext = [standardize_color(c, standard_colors = standard_colors_ext) for c in standard_colors_ext]

color_counts_int = cars_train['int_col'].value_counts()
standard_colors_int = list(color_counts_int[color_counts_int>500].keys())
standard_colors_int = [standardize_color(c, standard_colors = standard_colors_int) for c in standard_colors_int]

standard_colors = list(set(standard_colors_int + standard_colors_ext))
print(standard_colors)
for col in ['ext_col', 'int_col']:
    cars_train[col] = cars_train[col].apply(lambda x: standardize_color(x, standard_colors = standard_colors))
    cars_test[col] = cars_test[col].apply(lambda x: standardize_color(x, standard_colors = standard_colors))

['gold', 'black', 'gray', 'unknown', 'silver', 'brown', 'beige', 'yellow', 'orange', 'red', 'blue', 'white', 'green']


In [24]:
# use the gemini api to ask about which standard color is closest to a given color
model = genai.GenerativeModel("gemini-1.5-flash-002", generation_config = {"temperature":0.05})
color_dict = {col:col for color in standard_colors}
color_counts_ext = cars_train['ext_col'].value_counts()
color_counts_int = cars_train['int_col'].value_counts()
all_colors = list(set(list(color_counts_ext.keys())+list(color_counts_int.keys())))
for color in all_colors:
    if color in color_dict or color=='unknown': continue
    done = False
    while not done:
        try:
            prompt = f"Of the colors {standard_colors},which is the closest to {color}? If none are at all similar, just respond with {color}. Give your answer as just the color without any other words or punctuation. Make sure your answer is either a color from the list or {color} itself."
            answer = model.generate_content(prompt).text
            color_dict[color] = only_alphanum.sub("", answer).lower()
            print(color, color_dict[color])
            time.sleep(4)
            done = True
        except:
            time.sleep(4)
print(color_dict)


cloud white
richgarnetmetallic red
scarletember red
hotspur red
darkgraphitemetallic gray
cc cc
grigionimbus gray
brandy brown
slate gray
deepcypress green
satinsteelmetallic silver
linen beige
metallic silver
saharatan beige
platinum silver
riftmetallic silver
charcoal gray
amber gold
granitecrystalmetallicclearcoat gray
graphitewgunmetal gray
mediumstone beige
darkgalvanized gray
tempest gray
camel brown
iridiummetallic silver
biancoisis white
platinumquartzmetallic silver
darksapphire black
macchiato beige
purple purple
sandstone beige
portland gray
ametrinmetallic ametrinmetallic
granitecrystalclearcoatmetallic gray
maroon red
lightslate gray
mocha brown
billetclearcoatmetallic silver
moonlightcloud silver
neroade black
verde green
sandstonemetallic beige
black gray
magneticmetallic silver
nero black
customcolor pleaseprovidethevalueofcustomcolor
bronzedunemetallic brown
darkmattermetallic black
majesticplummetallic purple
gideon gideon
maximumsteelmetallic gray
navypier blue
shale

AttributeError: 'int_col' is not a valid function for 'Series' object

In [29]:
for color in standard_colors:
    color_dict[color] = color

In [45]:
color_dict

{'int_col': 'int_col',
 'cloud': 'white',
 'richgarnetmetallic': 'red',
 'scarletember': 'red',
 'hotspur': 'red',
 'darkgraphitemetallic': 'gray',
 'cc': 'cc',
 'grigionimbus': 'gray',
 'brandy': 'brown',
 'slate': 'gray',
 'deepcypress': 'green',
 'satinsteelmetallic': 'silver',
 'linen': 'beige',
 'metallic': 'silver',
 'saharatan': 'beige',
 'platinum': 'silver',
 'riftmetallic': 'silver',
 'charcoal': 'gray',
 'amber': 'gold',
 'granitecrystalmetallicclearcoat': 'gray',
 'graphitewgunmetal': 'gray',
 'mediumstone': 'beige',
 'darkgalvanized': 'gray',
 'tempest': 'gray',
 'camel': 'brown',
 'iridiummetallic': 'silver',
 'biancoisis': 'white',
 'platinumquartzmetallic': 'silver',
 'darksapphire': 'black',
 'macchiato': 'beige',
 'purple': 'purple',
 'sandstone': 'beige',
 'portland': 'gray',
 'ametrinmetallic': 'ametrinmetallic',
 'granitecrystalclearcoatmetallic': 'gray',
 'maroon': 'red',
 'lightslate': 'gray',
 'mocha': 'brown',
 'billetclearcoatmetallic': 'silver',
 'moonlightcl

In [46]:
for col in ['ext_col', 'int_col']:
    cars_train[col] = cars_train[col].apply(lambda x: color_dict[x])
    cars_test[col] = cars_test[col].apply(lambda x: color_dict[x])

Indicate that infrequent categorical values are probably mistakes or typos:

In [ ]:
threshold = 25
for cat in cat_types:
    value_counts = cars_train[cat].value_counts().to_dict()
    cars_train[cat] = cars_train[cat].apply(lambda x: x if value_counts[x] > threshold else "unknown")
    cars_test[cat] = cars_test[cat].apply(lambda x: x if (x in value_counts) and (value_counts[x] > threshold) else "unknown")


We indicate which columns are categorical.

In [ ]:
cars_train = cars_train.astype({col: "category" for col in cat_types})
cars_test = cars_test.astype({col: "category" for col in cat_types})

Drop unwanted columns:

In [ ]:
drop_cols = []       #list of columns to drop
cars_train.drop(columns=drop_cols, inplace=True)
cars_test.drop(columns=drop_cols, inplace=True)

In [47]:
cars_train.to_csv("train_processed.csv", index=False)
cars_test.to_csv("test_processed.csv", index=False)

# 2- XGBoost and Random Forest

Import the needed libraries:

In [ ]:
import sklearn as sk
import xgboost as xgb
import multiprocessing

In [ ]:
X = cars_train.loc[:,cars_train.columns != 'price']
y = cars_train['price']
X

,id,brand,model,model_year,milage,ext_col,int_col,accident,clean_title,age,reliability,full_name,brand_model,body_style,msrp,adjusted_msrp,price_diff,adjusted_price_diff
0,0,mini,coopersbase,2007,213000,yellow,gray,0,1,17,0.215360,2007minicoopersbase,minicoopersbase,hatchback,17400,26417.533348,-13200,-22217.533348
1,1,lincoln,lsv8,2002,143250,silver,beige,1,1,22,-0.432958,2002lincolnlsv8,lincolnlsv8,suv,47525,83160.610150,-42526,-78161.610150
2,2,chevrolet,silverado2500lt,2002,136731,blue,gray,0,1,22,0.461481,2002chevroletsilverado2500lt,chevroletsilverado2500lt,truck,26360,46125.485188,-12460,-32225.485188
3,3,genesis,g9050ultimate,2017,19500,black,black,0,1,7,0.781159,2017genesisg9050ultimate,genesisg9050ultimate,sedan,66000,84760.619469,-21000,-39760.619469
4,4,mercedesbenz,metrisbase,2021,7388,black,beige,0,1,3,-0.469551,2021mercedesbenzmetrisbase,mercedesbenzmetrisbase,cargovan,39950,46410.875300,57550,51089.124700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,toyota,sequoialimited,2008,269885,gray,black,0,1,16,0.774392,2008toyotasequoialimited,toyotasequoialimited,suv,50375,73653.482208,-43175,-66453.482208
196,196,kia,sorentosx,2011,11649,black,black,0,1,13,0.546895,2011kiasorentosx,kiasorentosx,suv,23200,32467.389923,-2200,-11467.389923
197,197,mercedesbenz,metrisbase,2020,26000,black,black,0,1,4,-0.469551,2020mercedesbenzmetrisbase,mercedesbenzmetrisbase,cargovan,35995,43781.334838,3005,-4781.334838
198,198,jeep,wranglerunlimitedsahara,2019,62197,blackclearcoat,black,0,1,5,-0.473834,2019jeepwranglerunlimitedsahara,jeepwranglerunlimitedsahara,suv,37400,46050.796406,-3450,-12100.796406


In [ ]:
xgb_model = xgb.XGBRegressor(enable_categorical=True, tree_method = 'hist')
clf = sk.model_selection.GridSearchCV(xgb_model,{"max_depth": [2, 4, 6], "n_estimators": [50, 100, 200]}, verbose=1, n_jobs=2)
clf.fit(X,y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


KeyboardInterrupt: 

In [ ]:
print(clf.best_score_)
print(clf.best_params_)

-1.25976462007868
{'max_depth': 2, 'n_estimators': 50}
